<a href="https://colab.research.google.com/github/aditya-saurabh/Coursera_Capstone/blob/master/Bombay_Neighbourhood_Battle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center>**Battle of Neighbourhood**</center> #
#### compare various neighbourhoods of Mumbai and clustering them based on similarity of various localities. This could be pretty useful for someone who is planning to change locality.<br> Key techniques has been listed below: ####
#### <ul><li> Neighbourhood data is collected from <a href = https://www.mapsofindia.com/pincode/india/maharashtra/mumbai/>here</a><li> Foursquare API has been used to explore the nearby venue<li>Folium library has been used to generate interactive maps of the city<li>K Means clustering algorithm has been used to cluster data

In [0]:
# Importing libraries

In [0]:
import pandas as pd
from geopy.geocoders import Nominatim
import numpy as np
import requests
import folium
from pandas.io.json import json_normalize

In [46]:
#For four-square API call
CLIENT_ID = 'NXLQWZGXTP2WIFOLL5P3QMROZFNFEC4LE3JJCIVZGLLRNUF5' # your Foursquare ID
CLIENT_SECRET = 'H2RZXGB5WGWMMFNPXC3TXSAEETKXRQJCHJ5FBTD4SEZKRX54' # your Foursquare Secret
VERSION = '20191012' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NXLQWZGXTP2WIFOLL5P3QMROZFNFEC4LE3JJCIVZGLLRNUF5
CLIENT_SECRET:H2RZXGB5WGWMMFNPXC3TXSAEETKXRQJCHJ5FBTD4SEZKRX54


The data of various localities of Mumbai has been taken from the url below. The table is extracted and but into the pandas dataframe for further investigation.

In [0]:
url_loc = 'https://www.mapsofindia.com/pincode/india/maharashtra/mumbai/'
neigh_data = pd.read_html(url_loc)

## Refining the dataset ##

In [6]:
df = neigh_data[0]
type(df)

pandas.core.frame.DataFrame

In [7]:
df.shape

(183, 4)

In [8]:
df.head()

,Pincode Details,Pincode Details.1,Pincode Details.2,Pincode Details.3
0,Location,Pincode,State,District
1,A I staff colony,400029,Maharashtra,Mumbai
2,Aareymilk Colony,400065,Maharashtra,Mumbai
3,Agripada,400011,Maharashtra,Mumbai
4,Airport,400099,Maharashtra,Mumbai


In [9]:
# Putting appropriate column heading
df.columns = df.loc[0]
df.drop(index = 0, axis = 0, inplace = True)
df.head()

,Location,Pincode,State,District
1,A I staff colony,400029,Maharashtra,Mumbai
2,Aareymilk Colony,400065,Maharashtra,Mumbai
3,Agripada,400011,Maharashtra,Mumbai
4,Airport,400099,Maharashtra,Mumbai
5,Ambewadi,400004,Maharashtra,Mumbai


In [10]:
df.shape

(182, 4)

In [11]:
df.reset_index(inplace = True, drop = True)
df.head()

,Location,Pincode,State,District
0,A I staff colony,400029,Maharashtra,Mumbai
1,Aareymilk Colony,400065,Maharashtra,Mumbai
2,Agripada,400011,Maharashtra,Mumbai
3,Airport,400099,Maharashtra,Mumbai
4,Ambewadi,400004,Maharashtra,Mumbai


In [0]:
df.drop(labels = ['State', 'District'], axis = 1, inplace = True, )

In [13]:
df.head()

,Location,Pincode
0,A I staff colony,400029
1,Aareymilk Colony,400065
2,Agripada,400011
3,Airport,400099
4,Ambewadi,400004


In [14]:
df.describe(include='all')

,Location,Pincode
count,182,182
unique,182,67
top,Government Colony,400003
freq,1,7


In [0]:
# There are only 67 unique Pin Code, so aggregating Location together and grouping by Pincode
df = df.groupby('Pincode').agg(lambda col: ', '.join(col)).reset_index(drop = False)

In [16]:
df.head()

,Pincode,Location
0,400001,"Bazargate, M.P.t., Mumbai., Stock Exchange, Ta..."
1,400002,"Kalbadevi, Ramwadi, S. c. court, Shroff Mahaja..."
2,400003,"B P t colony, B.P.lane, Mandvi, Masjid, Noor B..."
3,400004,"Ambewadi, Charni Road, Chaupati, Girgaon, Madh..."
4,400005,"Asvini, Colaba, Holiday Camp, V.W.t.c."


In [0]:
latitude = []
longitude = []
geolocation = Nominatim(user_agent= 'four-square')
for zipcode in df.Pincode:
  location = geolocation.geocode(query = {'country': 'India', 'city' : 'Mumbai', 'postalcode' : str(zipcode)})
  
  if location:
    latitude.append(location.latitude)
    longitude.append(location.longitude)
  else:
    latitude.append('NaN')
    longitude.append('NaN')
  


In [18]:
df['Latitude'] = latitude
df['Longitude'] = longitude
df.head()

,Pincode,Location,Latitude,Longitude
0,400001,"Bazargate, M.P.t., Mumbai., Stock Exchange, Ta...",18.9388,72.8353
1,400002,"Kalbadevi, Ramwadi, S. c. court, Shroff Mahaja...",18.9388,72.8353
2,400003,"B P t colony, B.P.lane, Mandvi, Masjid, Noor B...",18.9388,72.8353
3,400004,"Ambewadi, Charni Road, Chaupati, Girgaon, Madh...",18.9388,72.8353
4,400005,"Asvini, Colaba, Holiday Camp, V.W.t.c.",18.9388,72.8353


In [19]:
df.describe()

,Pincode,Location,Latitude,Longitude
count,67,67,67.000000,67.000000
unique,67,67,37.000000,37.000000
top,400003,"Kapad Bazar, Mahim, Mahim Bazar, Mahim East, M...",18.938771,72.835335
freq,1,1,28.000000,28.000000


In [0]:
df = df[df['Latitude'] != 'NaN']

In [21]:
df.describe()

,Pincode,Location,Latitude,Longitude
count,63,63,63.000000,63.000000
unique,63,63,36.000000,36.000000
top,400003,"Kapad Bazar, Mahim, Mahim Bazar, Mahim East, M...",18.938771,72.835335
freq,1,1,28.000000,28.000000


In [0]:

df.reset_index(drop = True, inplace = True)

In [0]:
#Removing dublicate co-ordinates and replacing it with the first value
bool_data = df.duplicated(subset  = 'Latitude', keep = 'first')

In [0]:
df = df[~bool_data]

In [25]:
df.reset_index(inplace = True, drop = True)
df.head()

,Pincode,Location,Latitude,Longitude
0,400001,"Bazargate, M.P.t., Mumbai., Stock Exchange, Ta...",18.9388,72.8353
1,400011,"Agripada, Chinchpokli, Haines Road, Jacob Circle",19.0359,72.8915
2,400017,"Dharavi, Dharavi Road",19.0458,72.8554
3,400029,"A I staff colony, Santacruz P&t colony",19.078,72.8648
4,400049,Juhu,19.1096,72.834


In [26]:
df.shape

(36, 4)

## Exploratory Data Analysis

In [28]:
#Plotting the various localitites in Mumbai
bombay = folium.Map(zoom_start = 10, location= [19.07, 72.87], height = '50%')

for loc, lat, lon in zip(df.Location, df.Latitude, df.Longitude):
  folium.CircleMarker(
      [float(lat), float(lon)],
      radius = 10,
      popup = str(loc),
      color = 'red',
      fill_color = True
  ).add_to(bombay)

bombay

In [0]:
data = df.copy()

In [0]:
latitude = data['Latitude'][0]
longitude = data['Longitude'][0]

LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
  CLIENT_ID,
  CLIENT_SECRET,
  VERSION, latitude, longitude,
  radius, LIMIT)
  
results = requests.get(url).json()

In [0]:
results = results['response']['groups'][0]['items']

In [0]:
result = json_normalize(results)

In [33]:
result.head()

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.crossStreet,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.postalCode,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.venuePage.id
0,e-0-5263e1ba11d265711e8024bf-0,0,"[{'summary': 'This spot is popular', 'type': '...",5263e1ba11d265711e8024bf,Town House Cafe,"# 16, Near Sterling Cinema, Murzban Road, Azad...",Fort,18.938550,72.833464,"[{'label': 'display', 'lat': 18.93854995852303...",198,400001,IN,Mumbai,Mahārāshtra,India,"[# 16, Near Sterling Cinema, Murzban Road, Aza...","[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",0,[],NaN
1,e-0-4caa0096d971b1f7ccca23e1-1,0,"[{'summary': 'This spot is popular', 'type': '...",4caa0096d971b1f7ccca23e1,Royal China,Waudby Road,NaN,18.938715,72.832933,"[{'label': 'display', 'lat': 18.93871523915629...",253,NaN,IN,Mumbai,Mahārāshtra,India,"[Waudby Road, Mumbai, Mahārāshtra, India]","[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",0,[],NaN
2,e-0-4b0587d9f964a52023a422e3-2,0,"[{'summary': 'This spot is popular', 'type': '...",4b0587d9f964a52023a422e3,Sher-E-Punjab,Shahid Bhagat Singh Road,NaN,18.937944,72.837853,"[{'label': 'display', 'lat': 18.93794383535345...",280,400 001,IN,Mumbai,Mahārāshtra,India,"[Shahid Bhagat Singh Road, Mumbai 400 001, Mah...","[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",0,[],NaN
3,e-0-4c714f4fd7fab1f715d760c9-3,0,"[{'summary': 'This spot is popular', 'type': '...",4c714f4fd7fab1f715d760c9,Cafe Excelsior,"23/A, K Nayak Marg","Opposite New Excelsior Cinema, Fort",18.937701,72.833566,"[{'label': 'display', 'lat': 18.93770120763569...",221,400001,IN,Mumbai,Mahārāshtra,India,"[23/A, K Nayak Marg (Opposite New Excelsior Ci...","[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",0,[],NaN
4,e-0-4babe4fff964a520f8d23ae3-4,0,"[{'summary': 'This spot is popular', 'type': '...",4babe4fff964a520f8d23ae3,Chhatrapati Shivaji Maharaj Terminus,Dr. Dadabhai Naoroji Road,D.N.Road,18.940088,72.835257,NaN,146,400001,IN,Mumbai,Mahārāshtra,India,"[Dr. Dadabhai Naoroji Road (D.N.Road), Mumbai ...","[{'id': '4bf58dd8d48988d129951735', 'name': 'T...",0,[],32965357


In [0]:
features = ['venue.name', 'venue.location.address', 'venue.categories']

In [35]:
result = result[features]
result.head()

,venue.name,venue.location.address,venue.categories
0,Town House Cafe,"# 16, Near Sterling Cinema, Murzban Road, Azad...","[{'id': '4bf58dd8d48988d116941735', 'name': 'B..."
1,Royal China,Waudby Road,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C..."
2,Sher-E-Punjab,Shahid Bhagat Singh Road,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I..."
3,Cafe Excelsior,"23/A, K Nayak Marg","[{'id': '4bf58dd8d48988d16d941735', 'name': 'C..."
4,Chhatrapati Shivaji Maharaj Terminus,Dr. Dadabhai Naoroji Road,"[{'id': '4bf58dd8d48988d129951735', 'name': 'T..."


In [36]:
result.iloc[0]['venue.categories'][0]['name']

'Bar'

In [0]:
result['venue.type'] = result['venue.categories'].apply(lambda x : x[0]['name'])

In [38]:
result.head()

,venue.name,venue.location.address,venue.categories,venue.type
0,Town House Cafe,"# 16, Near Sterling Cinema, Murzban Road, Azad...","[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",Bar
1,Royal China,Waudby Road,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",Chinese Restaurant
2,Sher-E-Punjab,Shahid Bhagat Singh Road,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",Indian Restaurant
3,Cafe Excelsior,"23/A, K Nayak Marg","[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",Café
4,Chhatrapati Shivaji Maharaj Terminus,Dr. Dadabhai Naoroji Road,"[{'id': '4bf58dd8d48988d129951735', 'name': 'T...",Train Station


In [39]:
result.columns

Index(['venue.name', 'venue.location.address', 'venue.categories',
       'venue.type'],
      dtype='object')

In [40]:
result.drop(labels = ['venue.categories'], inplace=True, axis=1)
result.head()

,venue.name,venue.location.address,venue.type
0,Town House Cafe,"# 16, Near Sterling Cinema, Murzban Road, Azad...",Bar
1,Royal China,Waudby Road,Chinese Restaurant
2,Sher-E-Punjab,Shahid Bhagat Singh Road,Indian Restaurant
3,Cafe Excelsior,"23/A, K Nayak Marg",Café
4,Chhatrapati Shivaji Maharaj Terminus,Dr. Dadabhai Naoroji Road,Train Station


In [41]:
result.rename(columns={'venue.name':'name', 'venue.location.address':'address', 'venue.type':'type'}, inplace=True)
result.shape

(19, 3)

## Exploring neighbourhood

In [0]:
columns = ['name', 'address', 'type', 'latitude', 'longitude', 'location']
data_final = pd.DataFrame(columns = columns)

def get_locations (url):
  results = requests.get(url).json()
  try:
    results = results['response']['groups'][0]['items']
    result = json_normalize(results)
    result = result[features]
    result['venue.type'] = result['venue.categories'].apply(lambda x : x[0]['name'])
    result.drop(labels = ['venue.categories'], inplace=True, axis=1)
    result.rename(columns={'venue.name':'name', 'venue.location.address':'address', 'venue.type':'type'}, inplace=True)
    return(result)

  except:
    return data_final
  
  
  


In [0]:
#Calling Four Square API

LIMIT = 150
radius = 1000

for loc, lat, lon in zip(data.Location, data.Latitude, data.Longitude):
  url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
  CLIENT_ID,
  CLIENT_SECRET,
  VERSION, lat, lon,
  radius, LIMIT)
  
  result = get_locations(url)
  result['latitude'] = result.shape[0]*[lat]
  result['longitude'] = result.shape[0]*[lon]
  result['location'] = result.shape[0]*[loc]

  data_final = data_final.append(result, ignore_index=True )

In [51]:
len(data_final['type'].unique())

209

In [52]:
data_final.describe(include = 'all')

,name,address,type,latitude,longitude,location
count,12247,10161,12247,12247.000000,12247.000000,12247
unique,1538,1237,209,NaN,NaN,17
top,Cafe Coffee Day,Oberoi Mall,Indian Restaurant,NaN,NaN,"Aareymilk Colony, Nagari Niwara"
freq,307,128,1393,NaN,NaN,11730
mean,NaN,NaN,NaN,19.171291,72.898171,NaN
std,NaN,NaN,NaN,0.010837,0.009581,NaN
min,NaN,NaN,NaN,19.076136,72.818314,NaN
25%,NaN,NaN,NaN,19.170991,72.900116,NaN
50%,NaN,NaN,NaN,19.170991,72.900116,NaN
75%,NaN,NaN,NaN,19.170991,72.900116,NaN


In [53]:
data_final.head()

,name,address,type,latitude,longitude,location
0,Royal China,Waudby Road,Chinese Restaurant,19.170991,72.900116,"Aareymilk Colony, Nagari Niwara"
1,Town House Cafe,"# 16, Near Sterling Cinema, Murzban Road, Azad...",Bar,19.170991,72.900116,"Aareymilk Colony, Nagari Niwara"
2,Sher-E-Punjab,Shahid Bhagat Singh Road,Indian Restaurant,19.170991,72.900116,"Aareymilk Colony, Nagari Niwara"
3,Cafe Excelsior,"23/A, K Nayak Marg",Café,19.170991,72.900116,"Aareymilk Colony, Nagari Niwara"
4,Yazdani Bakery,Cawasji Patel Street,Bakery,19.170991,72.900116,"Aareymilk Colony, Nagari Niwara"


In [0]:
#One Hot Encoding
one_hot = pd.get_dummies(data_final['type'])

In [55]:
one_hot.head()

,ATM,Accessories Store,Airport,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Bed & Breakfast,Beer Bar,Bengali Restaurant,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Cafeteria,...,Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Track,Train,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Volleyball Court,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
one_hot['location'] = data_final['location']

In [0]:
neigh_bombay = one_hot.copy()

In [0]:
neigh_bombay_filtered = neigh_bombay.groupby(by = 'location').mean()

In [59]:
neigh_bombay_filtered.head()

,ATM,Accessories Store,Airport,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Bed & Breakfast,Beer Bar,Bengali Restaurant,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Cafeteria,...,Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Track,Train,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Volleyball Court,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
location,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Aareymilk Colony, Nagari Niwara",0.000341,0.000682,0.000341,0.000682,0.000512,0.004092,0.000853,0.003922,0.000341,0.002558,0.020972,0.002046,0.000341,0.000000,0.000341,0.006479,0.001876,0.020631,0.0,0.028474,0.000171,0.002558,0.000853,0.001194,0.001535,0.000000,0.003751,0.005797,0.002558,0.000682,0.000341,0.005797,0.004092,0.000341,0.000853,0.004092,0.000682,0.002899,0.000171,0.000341,...,0.000853,0.024382,0.000341,0.000682,0.001876,0.000512,0.022165,0.002217,0.020460,0.000512,0.000341,0.007673,0.004774,0.018244,0.000853,0.000341,0.002046,0.005115,0.000853,0.000341,0.004092,0.000341,0.000341,0.002728,0.001023,0.004263,0.001194,0.003069,0.001705,0.000341,0.001194,0.006138,0.016880,0.000341,0.000341,0.000341,0.000853,0.001194,0.003922,0.001023
"Airport, International Airport, Sahar P & t colony, Sahargaon",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.026316,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.026316,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
"Andheri East, Nagardas Road",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Borivali,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,...,0.000000,0.176471,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000

In [60]:
neigh_bombay_filtered.shape

(17, 209)

# Clustering the data by K-Means and importing sklearn.cluster

In [0]:
# Applying K Means with four labels

In [0]:
from sklearn.cluster import KMeans

In [0]:
X = neigh_bombay_filtered

In [0]:
kmeans = KMeans(n_clusters=4, random_state=0).fit(X)

In [0]:
label = kmeans.labels_

In [0]:
#adding labels to the cluster

In [0]:
neigh_bombay_filtered['labels'] = label

In [77]:
neigh_bombay_filtered.head()

,ATM,Accessories Store,Airport,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Bed & Breakfast,Beer Bar,Bengali Restaurant,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Cafeteria,...,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Track,Train,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Volleyball Court,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio,labels
location,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Aareymilk Colony, Nagari Niwara",0.000341,0.000682,0.000341,0.000682,0.000512,0.004092,0.000853,0.003922,0.000341,0.002558,0.020972,0.002046,0.000341,0.000000,0.000341,0.006479,0.001876,0.020631,0.0,0.028474,0.000171,0.002558,0.000853,0.001194,0.001535,0.000000,0.003751,0.005797,0.002558,0.000682,0.000341,0.005797,0.004092,0.000341,0.000853,0.004092,0.000682,0.002899,0.000171,0.000341,...,0.024382,0.000341,0.000682,0.001876,0.000512,0.022165,0.002217,0.020460,0.000512,0.000341,0.007673,0.004774,0.018244,0.000853,0.000341,0.002046,0.005115,0.000853,0.000341,0.004092,0.000341,0.000341,0.002728,0.001023,0.004263,0.001194,0.003069,0.001705,0.000341,0.001194,0.006138,0.016880,0.000341,0.000341,0.000341,0.000853,0.001194,0.003922,0.001023,2
"Airport, International Airport, Sahar P & t colony, Sahargaon",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.026316,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.026316,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2
"Andheri East, Nagardas Road",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2
Borivali,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,...,0.176471,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000

# Data visualisation

In [0]:
color = ['blue', 'yellow', 'green', 'orange']

In [79]:
neigh_bom_map = folium.Map(zoom_start = 10, location= [19.07, 72.87])

for loc in neigh_bombay_filtered.index:
  lat = df[df['Location'] == loc]['Latitude'].unique()[0]
  lon = df[df['Location'] == loc]['Longitude'].unique()[0]
  folium.CircleMarker(
      [float(lat), float(lon)],
      radius = 10,
      popup = str(loc),
      color = color[int(neigh_bombay_filtered.loc[loc]['labels'])],
      fill_color = True
  ).add_to(neigh_bom_map)

neigh_bom_map

## Result

### It could be concluded from the above map that the localities of Mumbai are not much different from each other, there are mainly two types of cluster.The localities in close proximity are more similar than from localities which are far away, which also supports our intuition. <br>However, there is some amount of distinction between South and North Mumbai.

